<a href="https://colab.research.google.com/github/rajagopal17/END_NLP/blob/main/END_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd
from __future__ import unicode_literals
import spacy
from spacy.tokens import doc
nlp=spacy.load('en')
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
parser = English()
#########################################
import torch
from torchtext import data
from torchtext import datasets
SEED = 1200
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Reverse text

In [3]:
def reverse(text):
      return " ".join(text.split()[::-1])

In [4]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True,dtype=torch.int)
LABEL = data.LabelField(dtype = torch.float)

In [5]:
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:06<00:00, 12.1MB/s]


In [7]:
for i in range(0,25000):
 vars(train_data.examples[i]).get('text').reverse()

In [8]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [9]:
#check the length for each category of data
print(len(train_data))

print(len(test_data))

print(len(valid_data))

print(' ')
# Check contents for train & valid data
print(train_data.fields)
print(test_data.fields)
print(valid_data.fields)




17500
25000
7500
 
{'text': <torchtext.data.field.Field object at 0x7f3c875225c0>, 'label': <torchtext.data.field.LabelField object at 0x7f3c87528ac8>}
{'text': <torchtext.data.field.Field object at 0x7f3c875225c0>, 'label': <torchtext.data.field.LabelField object at 0x7f3c87528ac8>}
{'text': <torchtext.data.field.Field object at 0x7f3c875225c0>, 'label': <torchtext.data.field.LabelField object at 0x7f3c87528ac8>}


In [10]:
#Create vocabulary
from torchtext import vocab
MAX_VOCAB_SIZE = 25_000
vec = "glove.6B.100d"
TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors=vec,
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                          
100%|█████████▉| 398305/400000 [00:22<00:00, 17519.23it/s]

In [11]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [12]:
print(len(train_iterator))
print(train_iterator.device)
print(len(test_iterator))
print(test_iterator.device)

for i in test_iterator:
  print(i)
  break

274
cuda
391
cuda

[torchtext.data.batch.Batch of size 64 from IMDB]
	[.text]:('[torch.cuda.IntTensor of size 36x64 (GPU 0)]', '[torch.cuda.IntTensor of size 64 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 64 (GPU 0)]


In [47]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        ##############

        
        
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)

      
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths): 
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu())
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        #######################
      
        output = self.fc1(hidden)
        output = self.fc2(hidden)
        output = self.fc3(output)
                
        #hidden = [batch size, hid dim * num directions]
        hidden = self.dropout(torch.contiguous(hidden[-1,:,:]))   
        return hidden



        ########################
        #hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))           
        #return self.fc(hidden)


In [48]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = False
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [49]:
print(model)

RNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (rnn): LSTM(100, 256, num_layers=3, dropout=0.2)
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [50]:

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

rnn_model = to_device(model, device)# Added new
for sentiments in train_iterator:
    print(type(sentiments))
    print(sentiments)
    break

<class 'torchtext.data.batch.Batch'>

[torchtext.data.batch.Batch of size 64]
	[.text]:('[torch.cuda.IntTensor of size 90x64 (GPU 0)]', '[torch.cuda.IntTensor of size 64 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 64 (GPU 0)]


In [51]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
PAD_IDX

1

In [52]:
print(TEXT.vocab.freqs.most_common(5))
print(TEXT.vocab.itos[0:2])


[('the', 201423), (',', 191307), ('.', 165092), ('a', 108926), ('and', 108828)]
['<unk>', '<pad>']


In [53]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,051,305 trainable parameters


In [54]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [55]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.9712, -0.8279,  0.0602,  ..., -0.7963, -0.1292,  0.2732],
        [ 0.1591,  0.7252,  0.2821,  ..., -0.7047,  1.7645, -0.5982],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3028, -0.1248,  0.2981,  ..., -0.6897, -0.1911,  0.2448],
        [ 0.4327, -0.3285, -0.2883,  ..., -0.9027, -0.0802,  0.2962],
        [-0.5353,  0.5894, -0.2740,  ...,  0.5245,  0.7023,  0.0339]],
       device='cuda:0')

In [56]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)
print(UNK_IDX)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3028, -0.1248,  0.2981,  ..., -0.6897, -0.1911,  0.2448],
        [ 0.4327, -0.3285, -0.2883,  ..., -0.9027, -0.0802,  0.2962],
        [-0.5353,  0.5894, -0.2740,  ...,  0.5245,  0.7023,  0.0339]],
       device='cuda:0')
0


In [57]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [58]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [59]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [60]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [61]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [62]:

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [63]:
N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

RuntimeError: ignored